# Amazonのレビューを集めるためのスクリプト
法的な問題については以下を参考にしました:  
https://pig-data.jp/news_columns/scrapinglaw/

In [1]:
import requests, re
from bs4 import BeautifulSoup

In [51]:
# レビューデータを取得して出力する関数
def yield_review_data():
    for page in [1, 2]:
        top_url = "https://www.amazon.co.jp/gp/bestsellers/computers/?pg={}".format(page)
        base_url = "https://www.amazon.co.jp{}/?pageNumber={}"
        res = requests.get(top_url)
        parser = BeautifulSoup(res.text, "html.parser")
        elems = parser.find_all("a", attrs={"class":"a-link-normal"})
        for elem in elems:
            href = elem.get("href")
            matched = re.match(r"^/product-reviews/([^/]+)$", href)
            if matched:
                for page_num in range(1, 100):
                    url = "https://www.amazon.co.jp" + href + "/?pageNumber={}".format(page_num)
                    res = requests.get(url)
                    parser = BeautifulSoup(res.text, "html.parser")
                    reviews = parser.find_all("div", attrs={"class":"review"})
                    if len(reviews) == 0:
                        break
                    for review in reviews:
                        # まずStarを取得
                        stars = review.find_all("i", attrs={"class": "review-rating"})
                        if len(stars) == 0:
                            continue
                        for class_name in stars[0]["class"]:
                            if class_name[:7] == "a-star-":
                                star = class_name[7:]

                        # つぎレビュータイトルと本文
                        title = review.find_all("a", attrs={"class": "review-title"})
                        text = review.find_all("span", attrs={"class": "review-text-content"})
                        if len(title) > 0 and len(text) > 0:
                            yield star, title[0].text.strip(), text[0].text.strip()

In [52]:
# レビューを集める
reviews = []
for star, title, text in yield_review_data():
    reviews.append({"star": star, "title": title, "text": text})
    if len(reviews) % 100 == 0:
        print(len(reviews))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [54]:
# とりあえずレビューデータをそのままCSVに保存
import pandas as pd
df = pd.DataFrame(reviews)
df.to_csv("amazon_reviews.csv")

# trainとtestに分けてレビューデータを保存

In [60]:
# テキストのカラムを一つにまとめる
df["text"] = df["title"] + " " + df["text"]
df

,star,title,text
0,1,本製品はとにかく不安定なので返品しWSR-2533DHPL2に乗り換えたけどそれも不安定,本製品はとにかく不安定なので返品しWSR-2533DHPL2に乗り換えたけどそれも不安定 W...
1,1,ランプつき出し安いが問題があり、ネット接続が不安定になることが多い(1.05で直ったか？),ランプつき出し安いが問題があり、ネット接続が不安定になることが多い(1.05で直ったか？) ...
2,5,交換が簡単でした,交換が簡単でした WHR-1166DHP3からの買い替えです。テレワークに伴い、ネットワーク...
3,4,ネット接続途切れる方、私は解決しました！,ネット接続途切れる方、私は解決しました！ 私も普通に説明書通りにネット接続設定したら、1日に...
4,1,謎の途切れ多発。買わないほうが無難,謎の途切れ多発。買わないほうが無難 謎の途切れが多発して困っています。勝手にルータが再起動し...
...,...,...,...
19603,4,狭額縁ディスプレイだと爪が画面にかかる…,狭額縁ディスプレイだと爪が画面にかかる… くっきりすっきりした画質です．価格相応に高画質なの...
19604,5,オンラインミーティングなら必要十分,オンラインミーティングなら必要十分 自分撮りと、手元の資料を映す2つのカメラを使っています。...
19605,5,満足です,満足です 昔買ったC920に比べたら格段に画質が良い。
19606,5,M1チップ搭載Macでも問題なし,M1チップ搭載Macでも問題なし Mac mini(M1)で使用するために購入。USBケーブ...


In [61]:
# trainとtestにsplit
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["star"], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test

(2864     初、トラックボール 実は人生初のトラックボールどんもんかと試して見たら、なかなか快適MX2S...
 7463     これよりも…【追記あり】 皆さんがレビューしてらっしゃる通り、コスパがいい。ゲーミングモニタ...
 14911    有線接続でも安定して使えてます PCゲームをよくやるのですが、家の都合上ルーターから有線接続...
 14525    届いたら容量違う！ 購入はおすすめしません。　128GBの物を購入したのに届いて確認すると2...
 15496    純正品であるがゆえの信頼性は確かだが… もう少しお値段お安くならないものでしょうかね…。コス...
                                ...                        
 11284    速度、電波強度に不満なし。Wi-Fi Easy Mesh対応が楽しみ バッファロー製Wi-F...
 11964           純正 純正品の入手しやすさでAmazonを使っています。品質については言わずもがな。
 5390     人それぞれ好み！！ ゲームをしていて調べているとハードタイプのマウスパッドがあるのを知り購入...
 860      買い間違いでした。 Type-C to Type-C ケーブルでないと、PDP充電できないと...
 15795    最高のUSB-Cケーブル デスクでのPCの充電に使用しています。使う環境にもよりますが、私の...
 Name: text, Length: 17647, dtype: object,
 16824    バランスよくていいね 重低音も高音どちらも丁度いいしソフトウェアで自分でカスタマイズできるか...
 11843    コスパ最悪インク少なすぎ バッハのフルートソナタ1〜6番印刷するのに黒一色を二つ使いました。...
 7109     買ってよかったです 充電速度が速くて助かってます。コードの長さも長過ぎず短過ぎず丁度いいです...
 3971     初めてのウェブカメラ 概ね満足のいくウェブカメラでした。リモート会議で利用する上で残念な点は...
 7672     永久保証に期待していたが… 一眼カメラでの撮影の為、32GBのカードではすぐに一杯になるので.

In [62]:
# 保存
pd.DataFrame({"rating": y_train, "text": X_train}).to_csv("amazon_reviews_train.csv")
pd.DataFrame({"rating": y_test, "text": X_test}).to_csv("amazon_reviews_test.csv")